In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely import geometry, Polygon

IMPORT_FOLDER = Path('P:\moonshot2-casestudy\SFINCS\models')

# Loop over region folders and join the extent files
_combined_gdf = gpd.GeoDataFrame()
for _region in [f for f in IMPORT_FOLDER.iterdir() if f.is_dir()]:
    _extent_file = _region.joinpath('gis', 'region.geojson')
    assert _extent_file.is_file()
    _gdf = gpd.read_file(_extent_file)
    _combined_gdf = pd.concat([_combined_gdf, _gdf.to_crs(4326)], ignore_index=True)

# Extract the total extent spanning the concatenated regions
_total_extent = _combined_gdf.total_bounds



In [8]:
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper

# Read network from polygon
_polygon = Polygon(geometry.box(*_total_extent))
_network_section = NetworkSection(network_type=NetworkTypeEnum.DRIVE, road_types=[RoadTypeEnum.MOTORWAY, RoadTypeEnum.MOTORWAY_LINK, RoadTypeEnum.TRUNK, RoadTypeEnum.TRUNK_LINK, RoadTypeEnum.PRIMARY, RoadTypeEnum.PRIMARY_LINK, RoadTypeEnum.SECONDARY, RoadTypeEnum.SECONDARY_LINK, RoadTypeEnum.TERTIARY, RoadTypeEnum.TERTIARY_LINK], save_gpkg=True)
_config_data = NetworkConfigData(network=_network_section)
_network = OsmNetworkWrapper.with_polygon(_config_data, _polygon)


c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:454: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = utils_graph.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:501: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid


In [18]:
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter

# Export the graph to a GeoPackage
_exporter = MultiGraphNetworkExporter(basename='TotalNetwork', export_types=['gpkg'])
_exporter.export(export_path=Path('C:Temp'), export_data=_network)


TypeError: NetworkExporterBase.export() got an unexpected keyword argument 'output_dir'